In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin

data_path = '/content/drive/MyDrive/GR5243/compas-scores-two-years.csv'
compas_data = pd.read_csv(data_path)

features = ['age', 'sex', 'race', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree']
target = 'two_year_recid'

data = compas_data[features + [target]]

numeric_features = ['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count']
categorical_features = ['sex', 'race', 'c_charge_degree']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(solver='liblinear', max_iter=1000))])

X = data.drop(target, axis=1)
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
accuracy_base = accuracy_score(y_test, y_pred)

# Define Fair Logistic Regression
class FairLogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, sensitive_index, C=1.0, max_iter=100, fairness_strength=10.0):
        self.C = C
        self.max_iter = max_iter
        self.fairness_strength = fairness_strength
        self.sensitive_index = sensitive_index

    def fit(self, X, y):
        n_features = X.shape[1]
        weights = np.zeros(n_features)
        intercept = 0
        learning_rate = 0.01

        sensitive_features = X[:, self.sensitive_index]
        for _ in range(self.max_iter):
            predictions = 1 / (1 + np.exp(-(X.dot(weights) + intercept)))
            errors = y - predictions
            weights += learning_rate * (X.T.dot(errors) - self.C * weights)

            sensitive_errors = errors * sensitive_features
            mean_sensitive_errors = np.mean(sensitive_errors)
            fairness_adjustment = self.fairness_strength * mean_sensitive_errors

            weights[self.sensitive_index] += learning_rate * fairness_adjustment
            intercept += learning_rate * np.mean(errors)

        self.coef_ = weights
        self.intercept_ = intercept
        return self

    def predict_proba(self, X):
        scores = X.dot(self.coef_) + self.intercept_
        probabilities = 1 / (1 + np.exp(-scores))
        return probabilities

    def predict(self, X):
        probabilities = self.predict_proba(X)
        return (probabilities >= 0.5).astype(int)

feature_names = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
race_index = np.where(feature_names == 'race_African-American')[0][0]

fair_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', FairLogisticRegression(sensitive_index=race_index, fairness_strength=1.0))
])

fair_pipeline.fit(X_train, y_train)
y_pred_fair = fair_pipeline.predict(X_test)
accuracy_fair = accuracy_score(y_test, y_pred_fair)

print(f"Baseline Accuracy: {accuracy_base}")
print(f"Fairness Adjusted Accuracy: {accuracy_fair}")


Baseline Accuracy: 0.6923503325942351
Fairness Adjusted Accuracy: 0.5698447893569845


In [ ]:
# Exploring different balances between fairness and accuracy by adjusting the fairness_strength parameter
fairness_strengths = [0.1, 1, 5, 10, 100]
results = []

for strength in fairness_strengths:
    fair_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', FairLogisticRegression(sensitive_index=race_index, fairness_strength=strength))
    ])
    fair_pipeline.fit(X_train, y_train)
    y_pred_fair = fair_pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_fair)
    results.append((strength, accuracy))

results_df = pd.DataFrame(results, columns=['Fairness Strength', 'Accuracy'])
results_df


,Fairness Strength,Accuracy
0,0.1,0.569845
1,1.0,0.569845
2,5.0,0.569845
3,10.0,0.569845
4,100.0,0.569845


In [ ]:
#This part is to see how changing the sensitive attribute might be impacting the model.

class FairLogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, sensitive_index, C=1.0, max_iter=100, fairness_strength=10.0):
        self.C = C
        self.max_iter = max_iter
        self.fairness_strength = fairness_strength
        self.sensitive_index = sensitive_index

    def fit(self, X, y):
        n_features = X.shape[1]
        weights = np.zeros(n_features)
        intercept = 0
        learning_rate = 0.01

        print("Initial Weights:", weights)
        print("Initial Intercept:", intercept)

        for i in range(self.max_iter):
            predictions = 1 / (1 + np.exp(-(X.dot(weights) + intercept)))
            errors = y - predictions
            weights += learning_rate * (X.T.dot(errors) - self.C * weights)

            sensitive_errors = errors * X[:, self.sensitive_index]
            mean_sensitive_errors = np.mean(sensitive_errors)
            fairness_adjustment = self.fairness_strength * mean_sensitive_errors

            weights[self.sensitive_index] += learning_rate * fairness_adjustment
            intercept += learning_rate * np.mean(errors)

            if i % 10 == 0:
                print(f"Iteration {i}: Weights[{self.sensitive_index}] Change due to Fairness: {learning_rate * fairness_adjustment}")

        self.coef_ = weights
        self.intercept_ = intercept
        return self

    def predict_proba(self, X):
        scores = X.dot(self.coef_) + self.intercept_
        probabilities = 1 / (1 + np.exp(-scores))
        return probabilities

    def predict(self, X):
        probabilities = self.predict_proba(X)
        return (probabilities >= 0.5).astype(int)

sensitive_attribute = 'sex_Female'  # Example of changing sensitive attribute
race_index = np.where(feature_names == sensitive_attribute)[0][0]

# Recreate and fit the fair pipeline using the updated index
fair_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', FairLogisticRegression(sensitive_index=race_index, fairness_strength=1.0))
])

fair_pipeline.fit(X_train, y_train)

# Predict and evaluate the fair model again
y_pred_fair = fair_pipeline.predict(X_test)
accuracy_fair_updated = accuracy_score(y_test, y_pred_fair)

accuracy_fair_updated

Initial Weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Initial Intercept: 0
Iteration 0: Weights[0] Change due to Fairness: -0.0009388435045181634
Iteration 10: Weights[0] Change due to Fairness: 0.0005939629680062763
Iteration 20: Weights[0] Change due to Fairness: 0.001629964277756625
Iteration 30: Weights[0] Change due to Fairness: 0.00015205638143293138
Iteration 40: Weights[0] Change due to Fairness: 0.0017477088813581643
Iteration 50: Weights[0] Change due to Fairness: 0.00024278364152195909
Iteration 60: Weights[0] Change due to Fairness: 0.0017746706109855366
Iteration 70: Weights[0] Change due to Fairness: 0.00025663415744765696
Iteration 80: Weights[0] Change due to Fairness: 0.0017741592959928543
Iteration 90: Weights[0] Change due to Fairness: 0.0002559043698248839


0.5698447893569845

In [ ]:
# Testing FairLogisticRegression directly outside the pipeline
fair_classifier = FairLogisticRegression(sensitive_index=race_index, fairness_strength=1.0)
X_train_preprocessed = preprocessor.fit_transform(X_train)
y_train_array = y_train.to_numpy()

fair_classifier.fit(X_train_preprocessed, y_train_array)
y_pred_fair_direct = fair_classifier.predict(X_train_preprocessed)
accuracy_fair_direct = accuracy_score(y_train_array, y_pred_fair_direct)

print("Direct Fair Classifier Accuracy:", accuracy_fair_direct)


Initial Weights: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Initial Intercept: 0
Iteration 0: Weights[0] Change due to Fairness: -0.0009388435045181634
Iteration 10: Weights[0] Change due to Fairness: 0.0005939629680062763
Iteration 20: Weights[0] Change due to Fairness: 0.001629964277756625
Iteration 30: Weights[0] Change due to Fairness: 0.00015205638143293138
Iteration 40: Weights[0] Change due to Fairness: 0.0017477088813581643
Iteration 50: Weights[0] Change due to Fairness: 0.00024278364152195909
Iteration 60: Weights[0] Change due to Fairness: 0.0017746706109855366
Iteration 70: Weights[0] Change due to Fairness: 0.00025663415744765696
Iteration 80: Weights[0] Change due to Fairness: 0.0017741592959928543
Iteration 90: Weights[0] Change due to Fairness: 0.0002559043698248839
Direct Fair Classifier Accuracy: 0.5905730129390019
